#Data Collection for Tornado Model

In [32]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [33]:
# To avoid display of warnings in Jupyter Notebook:
import warnings
warnings.filterwarnings('ignore')

In [34]:
# Link to all the csv files:
link = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
response = requests.get(link)
html = response.text
print(html)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<html>
 <head>
  <title>Index of /pub/data/swdi/stormevents/csvfiles</title>
 </head>
 <body>
<h1>Index of /pub/data/swdi/stormevents/csvfiles</h1>
  <table>
   <tr><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>
   <tr><th colspan="4"><hr></th></tr>
<tr><td><a href="/pub/data/swdi/stormevents/">Parent Directory</a></td><td>&nbsp;</td><td align="right">  - </td><td>&nbsp;</td></tr>
<tr><td><a href="Storm-Data-Bulk-csv-Format.pdf">Storm-Data-Bulk-csv-Format.pdf</a></td><td align="right">2020-07-17 13:10  </td><td align="right">161K</td><td>&nbsp;</td></tr>
<tr><td><a href="Storm-Data-Export-Format.pdf">Storm-Data-Export-Format.pdf</a></td><td align="right">2020-07-17 09:17  </td><td align="right">163K</td><td>&nbsp;</td></tr>
<tr><td><a href="StormEvents_details-ftp_v1.0_d1950_c20170120.csv.gz">StormEvents_details-f

In [35]:
# Making a soup out of the html and creating the list of file names:
soup = BeautifulSoup(html)
elements = soup.findAll("a",{"class":""})

In [37]:
StormEventDetails_allyears = []

# The first 6 elements and the last 2 have to be discarded:
for element in elements[6:-2]:
    
    # Selecting only the storm events tables (one for each year)
    if element.attrs['href'].startswith('StormEvents_details'):
        
        filename = element.attrs['href']
        StormEventDetails_url = link + filename
        
        # Creating an iterator in order to load the file in chunks of 1000 elements:
        iter_csv = pd.read_csv(StormEventDetails_url, compression='gzip', iterator=True,
                               chunksize=1000)
        
        # Concatenating the different chunks into a single dataframe, 
        # selecting the tornado events only:
        StormEventDetails_allyears.append(
            pd.concat([chunk[chunk['EVENT_TYPE'].map(lambda x: x.lower())
                             == 'tornado'] for chunk in iter_csv]))

# Concatenating all the dataframes from the different years:
StormEventDetails = pd.concat(StormEventDetails_allyears)
        
print(StormEventDetails.shape)
StormEventDetails.head(2)

HTTPError: ignored

In [39]:
# Doing the same for the other two tables (locations and fatalities):
StormEventFatalities_allyears = []
StormEventLocations_allyears = []

for element in elements[6:-2]:
    if element.attrs['href'].startswith('StormEvents_fatalities'):
        filename = element.attrs['href']
        StormEventFatalities_url = link+filename
        iter_csv = pd.read_csv(StormEventFatalities_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventFatalities_allyears.append(pd.concat(iter_csv))

    elif element.attrs['href'].startswith('StormEvents_locations'):
        filename = element.attrs['href']
        StormEventLocation_url = link+filename
        iter_csv = pd.read_csv(StormEventLocation_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventLocations_allyears.append(pd.concat(iter_csv))
        
StormEventFatalities = pd.concat(StormEventFatalities_allyears)
StormEventLocations = pd.concat(StormEventLocations_allyears)

print(StormEventFatalities.shape)
print(StormEventLocations.shape)

StormEventFatalities.head(2)

HTTPError: ignored